This code assumes that the relevant OMOP tables have been loaded as Dask DataFrames, and are set up to be queried using a `dask-sql` context. The queries have been censored to mask table and column names, but they should translate to any OMOP environment.

First, we retrieve and save all concepts descended from "endometriosis" in the OMOP concept hierarchy.

In [ ]:
# c is a Context object from the dask_sql library
query = c.sql("""
    SELECT c.<concept IDs>, c.<concept names>, ca.<levels of separation> as separation
    FROM <concept table> AS c INNER JOIN <concept ancestor table> AS ca
    ON c.<concept IDs>=ca.<descendant concept IDs>
    WHERE ca.<ancestor concept IDs> = <concept ID for endometriosis>
    ORDER BY <concept IDs>
""")

concepts = query.compute()
concepts.to_csv("<concept filepath>", index=False)

Next, we use these concepts to retrieve the endometriosis patient population, and the non-endometriosis background population.

In [ ]:
# imports
import pandas as pd

# load endometriosis concepts
concepts = pd.read_csv("<concept filepath>")
concepts_str = ", ".join(concepts["<concept IDs>"].astype(str).tolist())

# set up query for patients with endometriosis 
query = c.sql(f"""
    SELECT <person ID>, <year of birth>, <gender concept ID> as gender, <race concept ID> as race, <ethnicity concept ID> as ethnicity
    FROM <person table>
    WHERE <person ID> IN (
      SELECT DISTINCT <person ID>
      FROM <condition occurrence table>
      WHERE <condition concept ID> IN ({concepts_str})
    ) ORDER BY <person ID>
""")

# process query
endo_pop = query.compute()
endo_person_ids = ", ".join(endo_pop["person_id"].astype(str).tolist())

# set up query for patients without endometriosis
query = c.sql(f"""
    SELECT <person ID>, <year of birth>, <gender concept ID> as gender, <race concept ID> as race, <ethnicity concept ID> as ethnicity
    FROM <person table>
    WHERE <person ID> IN (
      SELECT DISTINCT <person ID>
      FROM <condition occurrence table>
      WHERE <condition concept ID> NOT IN ({concepts_str})
    ) AND <person ID> NOT IN ({endo_person_ids})
    ORDER BY <person ID>
""")

# process query and combine DataFrames
background_pop = query.compute()
endo_pop["endo"] = [1] * endo_pop.shape[0]
background_pop["endo"] = [0] * background_pop.shape[0]
pop = pd.concat([endo_pop, background_pop], ignore_index=True)

# dictionary to convert gender concepts
gender_dict = {
    "<concept id>": "Male",
    "<concept id>": "Other",
    "<concept id>": "Female",
    "<concept id>": "Unknown"
}

# dictionary to convert race concepts
race_dict = {
    "<concept id>": "Asian",
    "<concept id>": "Black or African American",
    "<concept id>": "Other",
    "<concept id>": "White",
    "<concept id>": "Unknown",
    "<concept id>": "Native Hawaiian or Other Pacific Islander",
    "<concept id>": "American Indian or Alaska Native"
}

# dictionary to convert ethnicity concepts
ethnicity_dict = {
    "<concept id>": "Unknown",
    "<concept id>": "Hispanic or Latino",
    "<concept id>": "Not Hispanic or Latino"
}

# convert concepts
pop["gender"] = pop["gender"].map(gender_dict)
pop["race"] = pop["race"].map(race_dict)
pop["ethnicity"] = pop["ethnicity"].map(ethnicity_dict)

# calculate age
pop["<year of birth>"] = 2024 - pop["<year of birth>"]
pop.rename(columns={"<year of birth>": "age"}, inplace=True)

# set up query to retrieve healthcare utilization information
person_str = ", ".join(str(p) for p in pop["<person ID>"])
query = c.sql(f"""
    SELECT
        <person ID>,
        COUNT(DISTINCT <visit date>) AS num_visits,
        MIN(<visit date>) AS record_start,
        MAX(<visit date>) AS record_end
    FROM <visit occurrence table>
    WHERE <person ID> IN ({person_str})
    GROUP BY <person ID>
""")

# process query and merge information
visits = query.compute()
visits["record_duration"] = (visits["record_end"] - visits["record_start"]).apply(lambda x: x.days)
pop = pop.merge(visits, on="person_id")

# save population to disk
pop.to_feather("<population filepath>")